#### Numerical Linear Algebra: A Whirlwind Tour



In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import lu, lu_factor, lu_solve, solve, inv, det, svd
from scipy.special import comb

# make numpy print matrices in a more readable fashion
np.set_printoptions(precision=8, suppress=True, formatter={'float': '{: 0.8e}'.format}, linewidth=150)

Basic recipe for solving a linear system $Ax=b$ where $A$ is an $N\times N$ matrix and $x$ and $b$ are $N$-vectors:

Compute the *LU decomposition* of $A$:
$$ A = L\cdot U$$
where $L$ is a lower-triangular matrix with ones on the diagonal, and $U$ is an upper-triangular matrix.

With this decomposition, we can write the linear system as 
$$ L\cdot(U\cdot x) = b $$
First solve
$$ Ly = b$$
which is trivial since $L$ is lower-triangular (forward substitution), and then solve
$$ Ux = y$$
which is also trivial since $U$ is upper-triangular (back substitution).

This is especially useful if we have more than one RHS $b$ to solve with; the decomposition takes most of the time, but can be reused for multiple RHS.



Create a random (N,N) matrix:

In [2]:
N = 5
A = np.random.random((N,N))
A

array([[ 9.80211231e-01,  2.27603135e-01,  2.35291302e-01,  9.78760596e-01,  1.91777527e-01],
       [ 7.73573259e-01,  6.98721172e-02,  2.59023226e-01,  4.80220206e-01,  7.46047570e-01],
       [ 7.96732117e-01,  3.49676922e-01,  5.30008241e-01,  5.52383999e-01,  9.71622671e-01],
       [ 3.60024610e-01,  2.96912835e-02,  2.22795678e-01,  2.74912743e-01,  7.65489579e-01],
       [ 3.00373687e-01,  9.34050686e-01,  4.30205637e-01,  5.85936330e-01,  5.10059102e-01]])

Compute the LU decomposition of A. L,U are the LU decomposition, P is a permutation matrix.  
Use this if you need L and U explicitly:

In [3]:
P,L,U= lu(A)

What is the permutation matrix $P$? 

Imagine solving the system by hand. Start with writing 
$$ A \mathbf{1} = b $$

    Divide the zeroth row by $a_{00}$ to make it's diagonal 1. Then subtract the right amount of the zeroth row form each other row to make all of the $a_{i0}$'s zero, applying the same transformation to $\mathbf{1}$ and to $b$.

    Move on to the next row, divide by the new $a_{11}$, and subtract the right amount of row 1 from the other rows to make the entries in column 1 zero.
    
    Continue until $A$ has become the identity matrix. $\mathbf{1}$ has been turned into the solution to the system.

If the original $A$ has a zero on the diagonal, this method breaks down, but we can exchange rows to put that zero somewhere off the diagonal and bring a non-zero onto this row's diagonal. The process of permuting the order of the rows is called *partial pivoting*. 

It turns out that, even if no diagonal elements are zero, by choosing the largest element in a column (in absolute value) to be the pivot for the corresponding row, one minimizes the roundoff error in the algorithm. 

(Choosing the largest remaining element overall and interchanging both rows and columns, *full pivoting*, is even better, but partial pivoting is almost as good and is sufficient to render the algorithm stable.)

Without pivoting, the algorithm is unstable to roundoff error. *Always use an algorithm which using pivoting.*

The algorithm just described is known as Gauss-Jordan elimination, not LU decomposition, but pivoting is essential to all direct linear solution algorithms.

This permutation in rows is the one returned by `lu`.

In [4]:
print(L,"\n\n",U)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.06437712e-01  1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 8.12816760e-01  1.90531554e-01  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.89190365e-01 -1.26980808e-01  4.39160485e-01  1.00000000e+00  0.00000000e+00]
 [ 3.67292884e-01 -6.23689086e-02  5.86661996e-01 -8.61921998e-01  1.00000000e+00]] 

 [[ 9.80211231e-01  2.27603135e-01  2.35291302e-01  9.78760596e-01  1.91777527e-01]
 [ 0.00000000e+00  8.64304502e-01  3.58103509e-01  2.86007172e-01  4.51291236e-01]
 [ 0.00000000e+00  0.00000000e+00  2.70529509e-01 -2.97662409e-01  7.29757462e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.25169237e-01  3.31523279e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  5.80823837e-01]]


In [5]:
P

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

Verify that the decomposition is correct

In [6]:
A  - P @ L @ U

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.38777878e-17,  0.00000000e+00, -5.55111512e-17,  1.11022302e-16],
       [ 1.11022302e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.55111512e-17,  6.93889390e-18, -2.77555756e-17,  0.00000000e+00,  0.00000000e+00],
       [ 5.55111512e-17,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

A permutation matrix is orthogonal: inverse(P) = P.T  
We can permute the original matrix to compare with the permuted LU instead:

In [7]:
P.T @ A  -  L @ U

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.55111512e-17,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.11022302e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.38777878e-17,  0.00000000e+00, -5.55111512e-17,  1.11022302e-16],
       [ 5.55111512e-17,  6.93889390e-18, -2.77555756e-17,  0.00000000e+00,  0.00000000e+00]])

A more compact representation is returned by `lu_factor` which packs a single array with L and U, implicitly storing the unit diagonal of L.  
Use this if you want to compute with the LU decomposition but don't need the $L$ and $U$ matrices explicitly:

In [8]:
LU, piv = lu_factor(A)
LU

array([[ 9.80211231e-01,  2.27603135e-01,  2.35291302e-01,  9.78760596e-01,  1.91777527e-01],
       [ 3.06437712e-01,  8.64304502e-01,  3.58103509e-01,  2.86007172e-01,  4.51291236e-01],
       [ 8.12816760e-01,  1.90531554e-01,  2.70529509e-01, -2.97662409e-01,  7.29757462e-01],
       [ 7.89190365e-01, -1.26980808e-01,  4.39160485e-01, -1.25169237e-01,  3.31523279e-01],
       [ 3.67292884e-01, -6.23689086e-02,  5.86661996e-01, -8.61921998e-01,  5.80823837e-01]])

In [9]:
LU

array([[ 9.80211231e-01,  2.27603135e-01,  2.35291302e-01,  9.78760596e-01,  1.91777527e-01],
       [ 3.06437712e-01,  8.64304502e-01,  3.58103509e-01,  2.86007172e-01,  4.51291236e-01],
       [ 8.12816760e-01,  1.90531554e-01,  2.70529509e-01, -2.97662409e-01,  7.29757462e-01],
       [ 7.89190365e-01, -1.26980808e-01,  4.39160485e-01, -1.25169237e-01,  3.31523279e-01],
       [ 3.67292884e-01, -6.23689086e-02,  5.86661996e-01, -8.61921998e-01,  5.80823837e-01]])

The object `piv` is a one-dimensional array storing the permutation in the form  
swap row 0 with piv[0], then swap row 1 with piv[1], ...

In [10]:
piv

array([0, 4, 2, 4, 4], dtype=int32)

We can convert this to a permutation vector by swapping columns of the identity matrix accordingly:

In [11]:
def pivToPMatrix(piv):
    P = np.eye(N)
    for i, p in enumerate(piv):
        P[:,[i,p]] = P[:,[p,i]]
    return P

PP = pivToPMatrix(piv)
PP

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

and verify that we get the correct result

In [12]:
np.all((PP - P == 0))

True

Verify that the L and U are packed in LU as advertised:

In [13]:
LL = np.tril(LU, k=-1) + np.eye(N)  # take everything below the diagonal and add unit diagonal
UU = np.triu(LU)                    # take the diagonal and above
print( np.all((LL-L == 0)), np.all((LL-L == 0)) )

True True


We can then use `lu_solve((lu, piv), b)` to solve with RHS `b`:

In [14]:
b = np.random.random(N)
b

array([ 1.66668173e-01,  7.04756410e-02,  6.03795006e-01,  9.68366945e-02,  7.43610748e-01])

In [15]:
x = lu_solve((LU,piv), b)
x

array([-2.61072865e-01,  9.71419343e-02,  2.22254547e+00, -5.00182928e-02, -3.83385725e-01])

and check to see that the solution is correct

In [16]:
A @ x - b

array([-5.55111512e-17,  5.55111512e-17,  2.22044605e-16, -5.55111512e-17,  0.00000000e+00])

We can compute the inverse of A by solving a set of N systems with RHS given by the identity matrix:

In [17]:
b = np.eye(N)
x = np.zeros_like(A)
for i in range(N):
    x[:,i] = lu_solve((LU,piv), b[i])
    
# check that inverse(A) @ A is close to the identity matrix
print(A@x)
np.allclose(A@x - np.eye(N), 0)

[[ 1.00000000e+00  2.34491479e-16  1.00588485e-16 -2.73658390e-17  5.40591966e-18]
 [ 2.35425557e-16  1.00000000e+00  1.79460742e-16 -2.02493837e-16 -1.18611247e-17]
 [ 3.71083708e-16  1.37791732e-16  1.00000000e+00 -8.48626475e-16 -7.83018809e-17]
 [ 4.29454252e-17  1.73463158e-16  6.53637145e-18  1.00000000e+00  3.46029052e-17]
 [ 4.11559321e-17  4.00291556e-16 -5.96392743e-17 -3.33837804e-16  1.00000000e+00]]


True

Solving for multiple RHS's is automated by `solve`.  
The inverse is then

In [18]:
b = np.eye(N)
Ainv = solve(A,b)
print(Ainv @ A)
np.allclose(A@Ainv - np.eye(N), 0)

[[ 1.00000000e+00  1.48983825e-16  1.29713153e-16 -1.37730363e-16  5.70666115e-17]
 [-2.63953945e-16  1.00000000e+00 -2.45455654e-17 -3.87614313e-16 -2.13277755e-16]
 [ 8.45676425e-17  1.47923588e-17  1.00000000e+00  4.59424998e-16 -3.78682841e-17]
 [ 3.66920397e-16 -8.06607627e-17 -1.13327818e-16  1.00000000e+00 -2.70747387e-17]
 [ 1.27860291e-16 -2.41802750e-17 -1.29339398e-16 -7.97914968e-17  1.00000000e+00]]


True

This is automated by `inv`:

In [19]:
Ainv = inv(A)
np.allclose(A@Ainv - np.eye(N), 0)

True

The determinant of an LU decomposition is just the product of the diagonal elements, but the determinant changes sign every time a row is swapped with another. Thus, we need to find whether the permutation of rows was even or odd. We can determine this from piv by detecting which entries correspond to a swap and counting them:

In [20]:
piv

array([0, 4, 2, 4, 4], dtype=int32)

In [21]:
def evenOddPerm(piv):
    q = np.sum(piv != np.arange(0,N))
    return 1 if q % 2 == 0 else -1

In [22]:
LU.diagonal().prod() * evenOddPerm(piv)

-0.016662613818674194

This is automated by det(A):

In [23]:
det(A)

-0.016662613818674194

All of this seems to work quite well...  
Let's try a more "difficult" matrix. The Hilbert matrix $H_{ij} = 1/(i+j-1)$ is such a matrix:
$$ H = \begin{bmatrix}
1 & \frac{1}{2} & \frac{1}{3} & \frac{1}{4} & \dots \\
\frac{1}{2} & \frac{1}{3} & \frac{1}{4} & \frac{1}{5} & \dots \\
\frac{1}{3} & \frac{1}{4} & \frac{1}{5} & \frac{1}{6} & \dots\\
\frac{1}{4} & \frac{1}{5} & \frac{1}{6} & \frac{1}{7} &\dots \\
\vdots & \vdots & \vdots & \vdots &\ddots \\
\end{bmatrix} $$

In [24]:
def hilbert(n):
    m = np.arange(1, n + 1) + np.arange(0, n)[:, np.newaxis]
    return 1/m

N = 5
H = hilbert(N)
H

array([[ 1.00000000e+00,  5.00000000e-01,  3.33333333e-01,  2.50000000e-01,  2.00000000e-01],
       [ 5.00000000e-01,  3.33333333e-01,  2.50000000e-01,  2.00000000e-01,  1.66666667e-01],
       [ 3.33333333e-01,  2.50000000e-01,  2.00000000e-01,  1.66666667e-01,  1.42857143e-01],
       [ 2.50000000e-01,  2.00000000e-01,  1.66666667e-01,  1.42857143e-01,  1.25000000e-01],
       [ 2.00000000e-01,  1.66666667e-01,  1.42857143e-01,  1.25000000e-01,  1.11111111e-01]])

Now take the inverse, multiply by the original H, and compute the error by subtracting the identity matrix
$$ E = H^{-1}H - \mathbf{I}$$

We can summarize the error by taking a matrix norm. The max norm is
$$ ||A||_\infty  = \max_{i}\sum_{i=1}^N |A_{ij}|$$
and the the 2-norm is
$$ ||A||_\infty  = \max_{i}\sum_{i=1}^N A_{ij}^2$$

In [25]:
def maxNorm(A):
    return np.max(np.sum(np.abs(A), axis=1))

In [26]:
Hinv = inv(H)
q = Hinv @ H
print(q)
print(f"max-norm of error: {maxNorm(q-np.eye(q.shape[0]))}")
print(f"  2-norm of error: {np.linalg.norm(q-np.eye(q.shape[0]),2)}")

[[ 1.00000000e+00 -2.00395256e-14 -9.35759406e-16 -1.42108547e-14 -1.33596837e-14]
 [ 4.05808720e-13  1.00000000e+00  6.74381186e-14  4.54747351e-13  2.04034320e-13]
 [ 6.29496455e-13  6.88079223e-13  1.00000000e+00  0.00000000e+00  1.06504928e-12]
 [-1.70681247e-12 -1.60930528e-12 -2.67868267e-12  1.00000000e+00 -1.67919999e-12]
 [ 1.58100200e-12  1.10781754e-12  1.33934134e-12  0.00000000e+00  1.00000000e+00]]
max-norm of error: 7.674000412200879e-12
  2-norm of error: 4.95295242752307e-12


All seems well here...  
Let's try increasing the rank of our Hilbert matrix and see what happens

In [27]:
print(f" N     2-norm   max-norm")
for i in range(5,16):
    H = hilbert(i)
    Hinv = inv(H)
    q = Hinv @ H
    print(f"{i:2d}  {np.linalg.norm(q-np.eye(q.shape[0]),2): .2e}  {maxNorm(Hinv@H- np.eye(i)):.3e}")

 N     2-norm   max-norm
 5   4.95e-12  7.674e-12
 6   1.52e-10  2.257e-10
 7   5.57e-09  1.153e-08
 8   3.02e-07  6.547e-07
 9   7.84e-06  1.500e-05
10   1.53e-04  3.827e-04
11   5.13e-03  1.470e-02
12   1.82e-01  4.207e-01
13   2.71e+00  7.074e+00
14   6.65e+00  1.410e+01
15   4.97e+00  9.091e+00


By $N=10$ we only have 3 significant figures left in our result; by $N=12$ we have complete garbage!  
What is happening here? Why can't we solve such a simple linear system!

The inverse of the Hilbert matrix is
$$ (H^{-1})_{ij} = (-1)^{i+j} (i+j-1) \binom{n+i-1}{n-j} \binom{n+j-1}{n-i} \binom{i+j-2}{i-1}^2 $$

Python uses arbitrary precision integers and this is an integer result, so the inverse we compute this way must be exact. Let's try multiplying $H$ by this version of $H^{-1}$

In [38]:
def hilbertInv(n):
    Hinv = np.zeros((n,n))
    for ii in range(n):
        i = ii + 1
        for jj in range(n):
            j = jj + 1
            Hinv[ii,jj] = (-1)**(i+j) * (i+j-1) * comb(n+i-1, n-j, exact=True) \
                    * comb(n + j -1, n-i, exact=True) * comb(i+j-2,i-1, exact=True)**2
    return Hinv

for i in range(5,13):
    H = hilbert(i)
    Hinv = hilbertInv(i)
    print(f"{i:2d}  {np.linalg.norm(H@Hinv-np.eye(i)):.4e}")


 5  6.2992e-12
 6  2.9104e-11
 7  5.5533e-09
 8  1.7413e-07
 9  4.0036e-06
10  1.6838e-04
11  5.0052e-03
12  1.1836e-01


The result is just about as bad! What is going on?  
Here I take the product of row 2 of $H$ and row 7 of $H^{-1}$, showing the
terms. You can see that there is a wide spread in matrix element absolute value
as well as significant cancellation:

In [54]:
def HinvElement(n,i,j):
    return (-1)**(i+j) * (i+j-1) * comb(n+i-1, n-j, exact=True) * comb(n + j -1, n-i, exact=True) * comb(i+j-2,i-1, exact=True)**2

def HElement(n,i,j):
    return 1/(i+j-1)

N = 15
c = 0
for i in range(1,N+1):
    a = HElement(N,2,i)
    b = HinvElement(N,i,7)
    print(f"{a: .13e} * {b: .13e} = {a*b: .13e}")
    c += a*b
print(f"{c:.13e}")

 5.0000000000000e-01 *  5.2378326000000e+09 =  2.6189163000000e+09
 3.3333333333333e-01 * -1.0266151896000e+12 = -3.4220506320000e+11
 2.5000000000000e-01 *  5.0418212644800e+13 =  1.2604553161200e+13
 2.0000000000000e-01 * -1.0890333931277e+15 = -2.1780667862554e+14
 1.6666666666667e-01 *  1.2932271543391e+16 =  2.1553785905652e+15
 1.4285714285714e-01 * -9.4836657984869e+16 = -1.3548093997838e+16
 1.2500000000000e-01 *  4.5959303484975e+17 =  5.7449129356219e+16
 1.1111111111111e-01 * -1.5328700637554e+18 = -1.7031889597283e+17
 1.0000000000000e-01 *  3.5990511705258e+18 =  3.5990511705258e+17
 9.0909090909091e-02 * -5.9984186175430e+18 = -5.4531078341300e+17
 8.3333333333333e-02 *  7.0569630794623e+18 =  5.8808025662186e+17
 7.6923076923077e-02 * -5.7285173207022e+18 = -4.4065517851555e+17
 7.1428571428571e-02 *  3.0526967301110e+18 =  2.1804976643650e+17
 6.6666666666667e-02 * -9.6096725468584e+17 = -6.4064483645723e+16
 6.2500000000000e-02 *  1.3541326718119e+17 =  8.4633291988245

We can take a smaller example to get further insight. Consider the $2\times2$ system
$$ \begin{bmatrix} 1 & 1\\ 1 & 1.0001\end{bmatrix} x = \begin{bmatrix} 2\\ 2\end{bmatrix} $$
The solution is

In [30]:
A = np.array([[1, 1], [1, 1.0001]])
b = np.array([2, 2])
x = solve(A, b)
print("A = ")
print(A)
print()
print(f"b = ", b)
print(f"x = ", x)

A = 
[[ 1.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  1.00010000e+00]]

b =  [2 2]
x =  [ 2.00000000e+00  0.00000000e+00]


If we now perturb $b_1$ by only 0.0001, the solution changes radically

In [31]:
b = np.array([2, 2.0001])
x = solve(A, b)
print("b = ", b)
print("x = ", x)

b =  [ 2.00000000e+00  2.00010000e+00]
x =  [ 1.00000000e+00  1.00000000e+00]


The second row vector of the $A$ is very nearly parallel to the first row vector. If it were precisely parallel, the matrix would be singular and there would be no solution to our system. If the rows are sufficiently close to parallel, the system becomes very sensitive to small changes in parameters, and thus very sensitive to round-off errors.

This $2\times2$ system can be quite easily computed in Python's double precision, but as we increase 
$N$ the number of floating point operations increases as $N^2$ as the numerical situation becomes increasingly problematic.

We can explore this phenomenon further by examining another matrix decomposition, the *singular value decomposition* or SVD. We can write *any* matrix as
$$ A = U\cdot S\cdot V^T $$
where $S$ is a diagonal matrix containing the *singular values*, and $U$ and $V$ are orthogonal matrices.

If we think of multiplication by the $N\times M$ matrix $A$ as a map $A: \mathbb{R}^N\rightarrow \mathbb{R}^M$, multiplying a vector $x$ by $V^T$ rotates $x$, multiplying by $S$ stretches along the cardinal axes, and then multiplying by $U$ is another rotation.

The map $A$ acting on a vector in $\mathbb{R}^N$ may only be able to reach a lesser-dimensional subspace of $\mathbb{M}$ known as the range of $A$. The dimension of the range of $A$ is known as the *rank* of $A$. The rank of $A$ is the number of linearly independent columns and rows of $A$. The rank of $A$ is at least 1 and at most $\min(M,N)$.

Non-zero vectors in $\mathbb{N}$ which are mapped to zero by $A$ live in the *nullspace* of $A$ whose dimension is the *nullity* of $A$. The rank-nullity theorem says that the rank plus nullity of $A$ equals $N$.

For $A$ a square $N\times N$ matrix, if $A$'s rank is $N$, then $A$ is invertable and $Ax=b$ has a solution for any $b$, and only $x=0$ is mapped to $b=0$. If $A$ has rank less than $N$, then most vectors $b$ yeild no solution, but some have multiple solutions.

In the SVD, the columns of $U$ which correspond to non-zero elements of $S$ are an orthonormal basis for the range of $A$, and the columns of $V$ which correspond to zero elements of $S$ form an orthonormal basis for the null space of $A$.

We'll have more to say about SVD later on, but consider now square matrices. Because $U$ and $V$ are orthogonal, their inverse is their transpose, and thus the inverse of $A$ can be written
$$ A^{-1} = V \cdot \left[\textrm{diag}(1/s_i)\right]\cdot U^T $$
If one of the $s_i$ is zero, then $A$ is singular. If the range in values spanned by the set of $s_i$ spans more than the $10^{15}$ or so representable by a double-precision mantissa, then the matrix is *numerically singular*. Even if the range is smaller, it gives some idea of the loss of precision from roundoff error.
The ratio of the largest to the smallest singular value is known as the *condition number* 
$$ \textrm{condition number}(A) = \frac{\max_i{s_i}}{\min_i{s_i}} $$
As the condition number of $A$ grows, it becomes increasingly difficult to retain precision in solving a linear system (effectively inverting $A$). 

Let's examine the condition number of our $2\times 2$ system

In [32]:
U,S,V = svd(A)
print("S = ", S)
print(f"condition number = {np.max(S)/np.min(S)}")

S =  [ 2.00005000e+00  4.99987500e-05]
condition number = 40002.00007491522


Now look at the condition number of the $N\times N$ Hilbert matrix

In [33]:
for i in range(2,12):
    U,S,V = svd(hilbert(i))
    print(f"{i:2d}  condition number = {np.max(S)/np.min(S):.2e}")

 2  condition number = 1.93e+01
 3  condition number = 5.24e+02
 4  condition number = 1.55e+04
 5  condition number = 4.77e+05
 6  condition number = 1.50e+07
 7  condition number = 4.75e+08
 8  condition number = 1.53e+10
 9  condition number = 4.93e+11
10  condition number = 1.60e+13
11  condition number = 5.22e+14


Now go back and look at our results for inverting bits of the Hilbert matrix. The condition number
is important enough for numpy to give it its own function, `np.linalg.cond`

In [34]:
print(f" N     2-norm  cond. num.")
for i in range(5,16):
    H = hilbert(i)
    Hinv = inv(H)
    q = Hinv @ H
    c = np.linalg.cond(H)
    print(f"{i:2d}  {np.linalg.norm(q-np.eye(q.shape[0]),2): .2e}  {c:.2e}")

 N     2-norm  cond. num.
 5   4.95e-12  4.77e+05
 6   1.52e-10  1.50e+07
 7   5.57e-09  4.75e+08
 8   3.02e-07  1.53e+10
 9   7.84e-06  4.93e+11
10   1.53e-04  1.60e+13
11   5.13e-03  5.22e+14
12   1.82e-01  1.62e+16
13   2.71e+00  4.79e+17
14   6.65e+00  2.55e+17
15   4.97e+00  2.50e+17


Qeustions:   

    How does the time to solve a linear system grow with $N$?
    How does the time to take an inverse grow with $N$?

"In-place" inverses can be useful for large matrices

Tuesday we'll consider iterative methods and eigenvalue problems